In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.applications import DenseNet169
from keras.applications.densenet import preprocess_input
from data import *

Using TensorFlow backend.


In [2]:
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [3]:
STEPS = 2000
EPOCHS = 32
size = 128
batchsize = 64
NCATS = 340

In [4]:
base_model = DenseNet169(input_shape=(size, size, 3), weights=None, include_top=False)
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dropout(0.5)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(64)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(NCATS, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
#print(model.summary())

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, preprocess_input=preprocess_input, channel = 3)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 128, 128, 3) (34000, 340)
Validation array memory 6.23 GB


In [7]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, preprocess_input=preprocess_input, channel = 3)

In [8]:
callbacks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    TensorBoard(log_dir='log/dense169', 
                       histogram_freq=0, write_graph=True, write_images=False),
    ModelCheckpoint('models/dense169_best', monitor='val_categorical_accuracy', mode='max',
                       verbose=0, save_weights_only=True, save_best_only=True)
]

In [9]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/32
2000/2000 [==============================] - 686s 343ms/step - loss: 4.4857 - categorical_crossentropy: 4.4857 - categorical_accuracy: 0.0982 - top_3_accuracy: 0.2060 - val_loss: 3.7429 - val_categorical_crossentropy: 3.7429 - val_categorical_accuracy: 0.2011 - val_top_3_accuracy: 0.3762
Epoch 2/32
2000/2000 [==============================] - 656s 328ms/step - loss: 3.5385 - categorical_crossentropy: 3.5385 - categorical_accuracy: 0.2183 - top_3_accuracy: 0.4005 - val_loss: 4.1790 - val_categorical_crossentropy: 4.1790 - val_categorical_accuracy: 0.1370 - val_top_3_accuracy: 0.2802
Epoch 3/32
2000/2000 [==============================] - 656s 328ms/step - loss: 3.1921 - categorical_crossentropy: 3.1921 - categorical_accuracy: 0.2791 - top_3_accuracy: 0.4809 - val_loss: 2.4391 - val_categorical_crossentropy: 2.4391 - val_categorical_accuracy: 0.4305 - val_top_3_accuracy: 0.6514
Epoch 4/32
2000/2000 [==============================] - 657s 329ms/step - loss: 2.9745 - categorical

2000/2000 [==============================] - 653s 326ms/step - loss: 1.8104 - categorical_crossentropy: 1.8104 - categorical_accuracy: 0.5901 - top_3_accuracy: 0.7694 - val_loss: 1.4433 - val_categorical_crossentropy: 1.4433 - val_categorical_accuracy: 0.6629 - val_top_3_accuracy: 0.8268
Epoch 29/32
2000/2000 [==============================] - 653s 327ms/step - loss: 1.8109 - categorical_crossentropy: 1.8109 - categorical_accuracy: 0.5932 - top_3_accuracy: 0.7702 - val_loss: 1.3572 - val_categorical_crossentropy: 1.3572 - val_categorical_accuracy: 0.6828 - val_top_3_accuracy: 0.8411
Epoch 30/32
2000/2000 [==============================] - 653s 326ms/step - loss: 1.8015 - categorical_crossentropy: 1.8015 - categorical_accuracy: 0.5930 - top_3_accuracy: 0.7704 - val_loss: 3.7328 - val_categorical_crossentropy: 3.7328 - val_categorical_accuracy: 0.2231 - val_top_3_accuracy: 0.3789
Epoch 31/32
2000/2000 [==============================] - 653s 327ms/step - loss: 1.7848 - categorical_crossen

In [12]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, preprocess_input=preprocess_input, channel=3)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

(112199, 3) (112199, 128, 128, 3)
Test array memory 20.54 GB


In [13]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

112199/112199 [==============================] - 161s 1ms/step


(112199, 3)

In [14]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

(112199, 3)

In [15]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

,key_id,word
0,9000003627287624,radio stereo television
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China roller_coaster river
3,9000038588854897,mountain tent bridge
4,9000052667981386,campfire fireplace crown


In [16]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/densenet.csv', 'quickdraw-doodle-recognition', 
                        send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/densenet.csv.7z -m "submit"
